## Chat Models

Head to [Integrations](https://python.langchain.com/docs/modules/model_io/models/chat/) for documentation on built-in integrations with chat model providers.<br>
> 内置聊天模型集合

### Messages

> 1. The chat model interface is based around messages rather than raw text. 聊天模型使用**Message**而不是**原始文本**<br>
> 2. 现在支持的Message类型有: AIMessage, HumanMessage, SystemMessage ,FunctionMessage, **ChatMessage**
> 3. **ChatMessage**: 可以是任一角色: AI , Human, System, Function. `ChatMessage` takes in an arbitrary role parameter
> 4. 大部分情况下你只需要处理: `HumanMessage`, `AIMessage`, and `SystemMessage`

### 调用方法一: LCEL

#### 1. **LangChain Expression Language**: 这意味着其支持以下方法: <br>
> invoke, ainvoke<br>
> stream, astream<br>
> batch, abatch<br>
> astream_log<br>

#### 2. 入参:<br>

> 聊天模型的输入为 List[BaseMessage], 或者可以强制转换为Message的类型: 例如 str -> HumanMessage.  例如: PromptValue(好像是一种中间值?)


In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
chat = ChatOpenAI()

In [5]:
from langchain.schema.messages import HumanMessage, SystemMessage

In [6]:
messages = [
    SystemMessage(content='You are a helpful assistant'), 
    HumanMessage(content='What is the purpose of model regularization?')
]

##### invoke

In [7]:
chat.invoke(messages)

AIMessage(content="The purpose of model regularization is to prevent overfitting in machine learning models. Overfitting occurs when a model performs well on the training data but fails to generalize well to unseen data. Regularization techniques add a penalty term to the model's objective function, which discourages complex or excessive model parameters. This helps to control the model's capacity and reduce the risk of overfitting. Regularization can improve the model's generalization ability and make it more robust and reliable in real-world scenarios.")

##### stream

In [9]:
for chunk in chat.stream(messages):
    print(chunk.content, end='', flush=True)

The purpose of model regularization is to prevent overfitting in machine learning models. Overfitting occurs when a model learns the details and noise in the training dataset too well, leading to poor performance on new, unseen data. Regularization techniques are used to add a penalty to the model's objective function, discouraging the model from learning complex relationships that may not generalize well. By doing so, regularization helps to simplify the model and reduce its sensitivity to the training data, improving its ability to generalize and perform well on new, unseen data.

##### batch

In [8]:
chat.batch([messages])

[AIMessage(content="The purpose of model regularization is to prevent overfitting in machine learning models. Overfitting occurs when a model performs well on the training data but fails to generalize well to unseen data. Regularization techniques help to reduce the complexity of a model by adding a penalty term to the loss function, discouraging the model from learning overly complex patterns and focusing on the most important features. This helps to improve the model's ability to generalize to new data and increases its overall performance.")]

##### ainvoke

In [10]:
await chat.ainvoke(messages)

AIMessage(content="The purpose of model regularization is to prevent overfitting and improve the generalization ability of a machine learning model. Overfitting occurs when a model learns the training data too well, resulting in poor performance on new, unseen data. Regularization techniques introduce additional constraints or penalties to the model's objective function to discourage complex or extreme parameter values. This helps to simplify the model and reduce its reliance on noise in the training data, making it more robust and better able to make accurate predictions on unseen data.")

##### astream

In [12]:
async for chunk in chat.astream(messages):
    print(chunk.content, end='', flush=True)

The purpose of model regularization is to prevent overfitting in machine learning models. Overfitting occurs when a model is too complex and learns the noise or random fluctuations in the training data, rather than the underlying patterns or relationships. Regularization techniques help to constrain the model's complexity, reducing the risk of overfitting and improving its generalization ability. Regularization achieves this by penalizing large parameter values or by adding constraints to the model during training. This encourages simpler and more generalizable models that can perform well on unseen data.

##### astream_log

In [13]:
async for chunk in chat.astream_log(messages):
    print(chunk)

RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '239d7a52-07e5-416b-b2ed-41623ee49d04',
            'logs': {},
            'streamed_output': []}})
RunLogPatch({'op': 'add',
  'path': '/streamed_output/-',
  'value': AIMessageChunk(content='')})
RunLogPatch({'op': 'add',
  'path': '/streamed_output/-',
  'value': AIMessageChunk(content='The')})
RunLogPatch({'op': 'add',
  'path': '/streamed_output/-',
  'value': AIMessageChunk(content=' purpose')})
RunLogPatch({'op': 'add',
  'path': '/streamed_output/-',
  'value': AIMessageChunk(content=' of')})
RunLogPatch({'op': 'add',
  'path': '/streamed_output/-',
  'value': AIMessageChunk(content=' model')})
RunLogPatch({'op': 'add',
  'path': '/streamed_output/-',
  'value': AIMessageChunk(content=' regularization')})
RunLogPatch({'op': 'add',
  'path': '/streamed_output/-',
  'value': AIMessageChunk(content=' is')})
RunLogPatch({'op': 'add',
  'path': '/streamed_output/-',
  'value': AIMessageC

### 调用方法二: \_\_call\_\_
#### **Message in Message out**

In [14]:
from langchain.schema import (AIMessage, HumanMessage, SystemMessage)

In [15]:
chat([HumanMessage(content='Translate this sentence from English to French: I love programming.')])

AIMessage(content="J'adore programmer.")

#### multiple message

In [16]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="I love programming.")
]

In [17]:
chat(messages)

AIMessage(content="J'adore la programmation.")

#### generate
>  Batch calls, richer outputs, 批量调用, 丰富的输出


> You can go one step further and generate completions for multiple sets of messages using generate <br>
> 你可以更进一步, 生成多个Message的补全.

> This returns an LLMResult with an additional message parameter. <br>
> 这将返回带有附加消息的LLMResult

> 除了返回的Message(例如 finish reason) 和 附加的关于API调用信息(例如总的token消耗数)

In [18]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love artificial intelligence.")
    ],
]
result = chat.generate(batch_messages)
result

LLMResult(generations=[[ChatGeneration(text="J'adore la programmation.", generation_info={'finish_reason': 'stop'}, message=AIMessage(content="J'adore la programmation."))], [ChatGeneration(text="J'adore l'intelligence artificielle.", generation_info={'finish_reason': 'stop'}, message=AIMessage(content="J'adore l'intelligence artificielle."))]], llm_output={'token_usage': {'prompt_tokens': 53, 'completion_tokens': 20, 'total_tokens': 73}, 'model_name': 'gpt-3.5-turbo'}, run=[RunInfo(run_id=UUID('bad5c268-9bc3-4be2-b4fb-b89d5ffdd40d')), RunInfo(run_id=UUID('0d46415a-ac2c-4e99-a09f-4191ed69148b'))])

> You can recover things like token usage from this LLMResult:<br>
> 你可以恢复东西,像token使用量

In [19]:
result.llm_output

{'token_usage': {'prompt_tokens': 53,
  'completion_tokens': 20,
  'total_tokens': 73},
 'model_name': 'gpt-3.5-turbo'}